In [7]:
#settings and import libraries.


#TensorFlow
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from matplotlib.pyplot import imshow
tf.test.gpu_device_name()

#Google Drive
#Notes. Colaboratory may requires you a permission to access Google Drive.
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/oMap
%pwd

#matplot
%matplotlib inline
tf.test.gpu_device_name()

#others
import cv2
import numpy as np
import random
import os
import shutil

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/oMap'
/content/drive/My Drive/oMap


In [0]:
#globals
num_classes = 2
width,height=300,300
shape = (width,height,3) #RGB,300*300

In [0]:
def clearFolder(path):
  if os.path.isdir(path):
    contents=os.listdir(path)
    for content in contents:
      shutil.rmtree(os.path.join(path,content))
  else:
    os.makedirs(path)

In [0]:
def postMap(all, white):
    post = 255 - (imgWhite - imgAll)
    post = cv2.cvtColor(post, cv2.COLOR_BGR2GRAY)
    ret, post = cv2.threshold(post, 200, 255, cv2.THRESH_BINARY)
    return post

  
def getPost(imgPost, comp):
    circles = cv2.HoughCircles(imgPost, cv2.HOUGH_GRADIENT, 1,
                               20, param1=50, param2=30, minRadius=95, maxRadius=125)
    imgPost = cv2.medianBlur(imgPost, 3)
    circles = np.uint16(np.around(circles))
    imgPost = cv2.cvtColor(imgPost, cv2.COLOR_GRAY2BGR)

    #create an img which only the position of posts is drawn.
    for i in circles[0, :]:
        cv2.circle(imgPost, (i[0], i[1]), 2, (0, 0, 255), 3)
        cv2.circle(imgPost, (i[0], i[1]), i[2], (0, 255, 0), 2)
    cv2.imwrite("detected{}.png".format(comp), imgPost)

    return circles[0, :]

  
def trim(img, centerX, centerY, width, height, name,save=True):
    leftTop = (centerX - int(width / 2), centerY - int(height / 2))
    imgTrim = img[leftTop[1]:leftTop[1] +
                  height, leftTop[0]:leftTop[0] + width]
    if save:
        cv2.imwrite(name, imgTrim)
    return imgTrim


def inflation(folder,out):
    files=os.listdir(folder)
    for file in files:
        if file[-4::]==(".png" or ".jpg"):
            img=cv2.imread(os.path.join(folder,file))
            for i in range(4):
                img=img.transpose(1,0,2)[:,::-1]
                cv2.imwrite(os.path.join(out,file)[:-4:]+"-{}.png".format(90*i),img)
                imgMirror=cv2.flip(img,1)
                cv2.imwrite(os.path.join(out,file)[:-4:]+"-{}-t.png".format(90*i),imgMirror)




In [0]:
for comp in (38, 39):
    imgAll = cv2.imread("all{0}.png".format(comp))
    imgWhite = cv2.imread("white{0}.png".format(comp))
    imgPost = postMap(imgAll, imgWhite)
    cv2.imwrite("thresh{}.png".format(comp),imgPost)
    posts = getPost(imgPost, comp)
    for i, post in enumerate(posts):
        trim(imgWhite, post[0], post[1], width, height,"Post/{0}post-{1}.png".format(comp, i))
        trim(imgWhite, random.randint(posts.min(axis=0)[0], posts.max(axis=0)[0]), random.randint(
                posts.min(axis=0)[1], posts.max(axis=0)[1]), width, height, "NOTPost/{0}not-{1}.png".format(comp, i))

    inflation("Post","Post水増し")
    inflation("NOTPost","NOTPost水増し")

In [0]:
num_classes = 2
shape = (300,300,3) #RGB,300*300

def cnn():
  model = Sequential()
  model.add(Conv2D(32,(3,3),padding="same",activation="relu",input_shape=shape))
  model.add(Conv2D(32,(3,3),padding="same",activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  
  return model
  

In [0]:
model = cnn()

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 300, 300, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 64)        0         
__________

In [0]:
model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
                  metrics=['accuracy'])

In [0]:
history=model.fit()